In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from time import sleep

In [3]:
import importlib
import numpy as np
import tensorflow as tf
import os
from config import CONFIG
MODELS = importlib.import_module(
    '.'.join(('musegan', CONFIG['exp']['model'], 'models')))
os.environ['CUDA_VISIBLE_DEVICES']

'1'

In [ ]:
def load_data():
    """Load and return the training data."""
    print('[*] Loading data...')

    # Load data from SharedArray
    if CONFIG['data']['training_data_location'] == 'sa':
        import SharedArray as sa
        x_train = sa.attach(CONFIG['data']['training_data'])

    # Load data from hard disk
    elif CONFIG['data']['training_data_location'] == 'hd':
        if os.path.isabs(CONFIG['data']['training_data']):
            x_train = np.load(CONFIG['data']['training_data'])
        else:
            filepath = os.path.abspath(os.path.join(
                os.path.realpath(__file__), 'training_data',
                CONFIG['data']['training_data']))
            x_train = np.load(filepath)

    # Reshape data
    x_train = x_train.reshape(
        -1, CONFIG['model']['num_bar'], CONFIG['model']['num_timestep'],
        CONFIG['model']['num_pitch'], CONFIG['model']['num_track'])
    print('Training set size:', len(x_train))

    return x_train

In [ ]:
x_train = load_data()
y_train = np.load('/home/rakova/project/multi_musegan/musegan/training_data/binary_data/binary_labels.npy')
x_train_big = np.load('/home/rakova/project/multi_musegan/musegan/training_data/binary_data_big/binary_train.npy')
x_test = x_train_big[-1024:]
y_train_big = np.load('/home/rakova/project/multi_musegan/musegan/training_data/binary_data_big/binary_labels.npy')
y_test = y_train_big[-1024:]

with tf.Session(config=CONFIG['tensorflow']) as sess:
    gan = MODELS.GAN(sess, CONFIG['model'])
    gan.init_all()
    # Load pretrained model if given
    if CONFIG['exp']['pretrained_dir'] is not None:
        gan.load(CONFIG['exp']['pretrained_dir'])
    gan.train(x_train, y_train, CONFIG['train'], x_test, y_test)